![HydroSAR Banner](./../NotebookAddOns/HydroSARbanner.jpg)

# Load HyP3 Data for Full Frame Processing

<img style="padding: 7px" src="../../Master/NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/>

### Franz J Meyer and Alex Lewandowski; University of Alaska Fairbanks

This notebook downloads data from an ASF-HyP3 subscription and prepares it for full frame processing with subsequent HydroSAR algorithms. Data are downloaded, extracted, and organized. DEM tiles are extracted and mosaicked to a consistent large-scale product for use in other HydroSAR workflows.
    
**Complete Full Frame Flood Mapping Sequence:**

This notebook is the first step in the **Full Frame Flood Mapping Sequence**, which consists of running the following notebooks in the order given:

1. **Load HyP3 Data:** Run this notebook (LoadHyP3Data-FullFrame.ipynb)
1. **Calculate HAND layer:** Run [Big_Hand_notebook.ipynb](./Big_Hand_notebook.ipynb)
1. **Perform Full Frame Surface Water Mapping:** Run [HYDRO30Workflow-v1.ipynb](./HYDRO30Workflow-v1.ipynb)

---
**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/hydrosar':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "hydrosar" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "hydrosar" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "hydrosar" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## Importing Relevant Python Packages

In this notebook we will use the following scientific libraries:

1. **[GDAL](https://www.gdal.org/)** is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.
1. **[NumPy](http://www.numpy.org/)** is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects.

**Our first step is to import them:**

In [ ]:
%%capture
from datetime import datetime
import glob
import json # for loads
import os
from pathlib import Path
import pytz
import re
import shutil
from tqdm import tqdm

import numpy as np
from osgeo import gdal

from IPython.display import HTML, display, clear_output, Markdown
    
from hyp3_sdk import Batch, HyP3

import opensarlab_lib as asfn

## Load Your Own Data Stack Into the Notebook

This notebook assumes that you've created your own data stack over your personal area of interest using the [Alaska Satellite Facility's](https://www.asf.alaska.edu/) value-added product system [HyP3](https://hyp3-docs.asf.alaska.edu/). HyP3 is an environment that is used by ASF to prototype value added products and provide them to users to collect feedback. 

This lab expects [Radiometric Terrain Corrected](https://media.asf.alaska.edu/uploads/RTC/rtc_atbd_v1.2_final.pdf) (RTC) image products as input, so be sure to select an RTC process when creating the subscription for your input data within HyP. Prefer a unique orbit geometry **(choose ascending or descending, not both)** to keep geometric differences between images low. 

We will retrieve HyP3 data via the HyP3 API. As both HyP3 and the Notebook environment sit in the [Amazon Web Services (AWS)](https://aws.amazon.com/) cloud, data transfer is quick and cost effective.

---

Before we download anything, create a working directory for this analysis and change into it. 

**Select or create a working directory for the analysis:**

In [ ]:
while True:
    print(f"Current working directory: {Path.cwd()}")
    data_dir = Path(input(f"\nPlease enter the name of a directory in which to store your data for this analysis."))
    if data_dir == Path('.'):
        continue
    if data_dir.is_dir():
        contents = data_dir.glob('*')
        if len(list(contents)) > 0:
            choice = asfn.handle_old_data(data_dir)
            if choice == 1:
                if data_dir.exists():
                    shutil.rmtree(data_dir)
                data_dir.mkdir()
                break
            elif choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        data_dir.mkdir()
        break

**Change into the analysis directory and create a folder in which to download your RTC products:**

In [ ]:
# Setting Paths
analysis_directory = Path.cwd()/data_dir
print(f"analysis_directory: {analysis_directory}")

products_path = analysis_directory/"rtc_products"
products_path.mkdir()

**Create a HyP3 object and authenticate**

In [ ]:
hyp3 = HyP3(prompt=True)

**List your Hyp3 RTC_GAMMA projects containing active products and select one:**

In [ ]:
my_hyp3_info = hyp3.my_info()
active_projects = dict()

for project in my_hyp3_info['job_names']:
    batch = Batch()
    try:
        batch = hyp3.find_jobs(name=project, job_type='RTC_GAMMA').filter_jobs(include_expired=False)
    except UserWarning:
        pass       
    if len(batch) > 0:
        active_projects.update({batch.jobs[0].name: batch})
        
if len(active_projects) > 0:
    display(Markdown("<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:darkred;'>Otherwise, you will rerun this code cell.</text>"))
    print('\nSelect a Project:')
    project_select = asfn.select_parameter(active_projects)
    display(project_select)
else:
    print("Found no active RTC_GAMMA jobs in any HyP3 projects") 

**Select a date range of products to download:**

In [ ]:
batch = project_select.value

display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
dates = asfn.get_job_dates(batch)
date_picker = asfn.gui_date_picker(dates)
date_picker

**Save the selected date range and remove products falling outside of it:**

In [ ]:
date_range = asfn.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")
batch = asfn.filter_jobs_by_date(batch, date_range)

**Gather the available paths and orbit directions for the remaining products:**

In [ ]:
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>"))
asfn.set_paths_orbits(batch)
paths = set()
orbit_directions = set()
for p in batch:
    paths.add(p.path)
    orbit_directions.add(p.orbit_direction)
paths.add('All Paths')
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))

---
**Select a path or paths (use shift or ctrl to select multiple paths):**

In [ ]:
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = asfn.select_mult_parameters(paths)
path_choice

In [ ]:
flight_path = path_choice.value
if flight_path:
    if flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

**Select an orbit direction:**

In [ ]:
if len(orbit_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = asfn.select_parameter(orbit_directions, 'Direction:')
direction_choice

**Save the selected orbit direction:**

In [ ]:
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

**Filter jobs by path and orbit direction:**

In [ ]:
batch = asfn.filter_jobs_by_path(batch, flight_path)
batch = asfn.filter_jobs_by_orbit(batch, direction)
print(f"There are {len(batch)} products to download.")

**Download the products, unzip them into the rtc_products directory, and delete the zip files:**

In [ ]:
print(f"\nProject: {batch.jobs[0].name}")
project_zips = batch.download_files(products_path)
for z in project_zips:
    asfn.asf_unzip(str(products_path), str(z))
    z.unlink()

## Migrate VV & VH images into folder `FullFrames`

In [ ]:
# Some Uitility Functions needed for Extracting the Relevant Information from the downloaded Zip Files
def get_tiff_paths(regex, pths):
    tiff_paths = list()
    paths = (Path.cwd()).glob(pths)
    for pth in list(paths):
        tiff_path = re.search(regex, str(pth))
        if tiff_path:
            tiff_paths.append(pth)
    return tiff_paths

with asfn.work_dir(products_path):
    regex = "/(\w{1,600}/){0,}\w{5,600}(_|-)V(V|H).(tif|tiff)$"
    pths = f"*/*.tif*"
    tiff_paths = get_tiff_paths(regex, pths)

In [ ]:
# Extract Relevant Files from Zips and Organize your Data
ff_path = Path(f"{analysis_directory}/FullFrames/")
if not ff_path.is_dir():
    ff_path.mkdir()
print('Copying VV and VH bands into folder FullFrames ...')
for pth in tqdm(tiff_paths):
    pol_regex = '(vv|VV|vh|VH)(?=.tif)'
    pol = re.search(pol_regex, str(pth))
    if pol:
        pol = pol.group(0)
    dt_regex = '\d{8}T\d{6}'
    dt = re.search(dt_regex, str(pth))
    if dt:
        dt = dt.group(0)
    outname = (f"{dt}_{pol}.tif")
    pth.rename(ff_path/outname)

## Mosaicking DEMs 

### Collect DEM files and Migrate into Folder `DEMs`

In [ ]:
# Look for DEM files in your Subscription --> Needed to facilitate HAND Calculation
dem_path = analysis_directory/'DEMS'
if not dem_path.is_dir():
    dem_path.mkdir()
dems_relative = f"*/*/*/*dem.tif"
print(dems_relative)
dems = Path().glob(dems_relative)
for d in dems:
    print(dem_path/d.name)
    d.rename(dem_path/d.name)

## Performing Mosaicking

**Writing Support Functions:**

In [ ]:
# Writing some Uitility Functions
def get_DEM_paths(pths):
    DEM_paths = []
    
    for pth in glob.glob(pths):
        DEM_paths.append(pth)
    return DEM_paths

def get_DEM_dates(paths):
    dates = []
    for pth in tiff_paths:
        date = pth.split("/")[-1].split("_")[3].split("T")[0]
        dates.append(date)
    return dates

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

**Identifying Predominant UTM zone:**

In [ ]:
# Check UTM Zones and Identify Dominant UTM Zone for your data stack

DEM_paths = list(dem_path.glob("*.tif"))

utm_zones = []
utm_types = []
print('Checking UTM Zones in the DEM data stack ...\n')
for dem in DEM_paths:
    info = (gdal.Info(str(dem), options = ['-json']))
    info = json.dumps(info)
    info = (json.loads(info))['coordinateSystem']['wkt']
    zone = info.split('ID')[-1].split(',')[1][0:-2]
    utm_zones.append(zone)
    typ = info.split('ID')[-1].split('"')[1]
    utm_types.append(typ)
print(f"UTM Zones:\n {utm_zones}\n")
print(f"UTM Types:\n {utm_types}")

# Identifying Predominant UTM Zone

utm_unique, counts = np.unique(utm_zones, return_counts=True)
a = np.where(counts == np.max(counts))
predominant_utm = utm_unique[a][0]
print(f"\nPredominant UTM Zone: {predominant_utm}")

**Reproject and Mosaic DEMs:**

In [ ]:
# Reproject (if needed) and Mosaic DEM Files in Preparation for Subsequent HAND Calculation
# print(DEM_paths)
reproject_indicies = [i for i, j in enumerate(utm_zones) if j != predominant_utm] #makes list of indicies in utm_zones that need to be reprojected
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproject_indicies)))
print('--------------------------------------------')
for k in reproject_indicies:
    temppath = f"{str(DEM_paths[k].parent)}/r{DEM_paths[k].name}"
    print(temppath)  

    cmd = f"gdalwarp -overwrite {DEM_paths[k]} {temppath} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominant_utm}"
#     print(cmd)
    !{cmd}
    
    DEM_paths[k].unlink()

print(' ')
print('--------------------------------------------')
print('Update DEM Paths')
print('--------------------------------------------')
DEM_paths = list(dem_path.glob("*.tif"))
for d in DEM_paths:
    print(d)


print(' ')
print('--------------------------------------------')
print('Build String of DEMs to Merge')
print('--------------------------------------------')
DEM_merge_paths = ""
for pth in DEM_paths:
    DEM_merge_paths = f"{str(pth)} {DEM_merge_paths}"
print(DEM_merge_paths)
   
print(' ')
print('--------------------------------------------')
print('Create DEM Mosaic')
print('--------------------------------------------')
output = f"{ff_path}/DEM-Mosaic.tif"
gdal_command = f"gdal_merge.py -o {output} {DEM_merge_paths}"
# print(f"\n\nCalling the command: {gdal_command}\n")
!{gdal_command}

## Cleanup Workspace

In [ ]:
# Cleaning up Temporary Files
shutil.rmtree(dem_path)
shutil.rmtree(products_path)

---

*LoadHyP3Data-v2-FullFrame.ipynb - Version 1.1.0 - November 2021*

*Version Changes:*

- *asf_notebook -> opensarlab_lib*
- *url_wdiget*
- *html -> markdown*